In [1]:
!conda activate Jatin_GPU_ENV

# Let's Fine tune an open sourse model for our use!!

## Basic Installations

In [13]:
import os
os.environ['LC_ALL'] = 'C.UTF-8'

In [3]:
!pip install nltk

In [4]:
# Basic Installations
!pip install transformers
!pip install mistral_common

In [5]:
!pip install autoawq

In [6]:
!pip install gradio

In [7]:
!pip install -U datasets

In [8]:
!pip install bitsandbytes

In [9]:
!pip install rouge_score

In [10]:
!pip install accelerate

In [11]:
!pip install evaluate

## Hugging Face Authentication

In [14]:
from huggingface_hub import notebook_login

notebook_login()

## MIstral 7-B Model

In [12]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


def model_formation(model_name_or_path):

  model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)

  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)

  return model, tokenizer

def tokenize(tokenizer, prompt):
  tokens = tokenizer(
  prompt,
  return_tensors='pt'
  ).input_ids.cuda()

  return tokens

def generate_output(model,tokens):
  # Generate output
  generation_output = model.generate(
    tokens,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
  )
  return generation_output

def decode_output(generated_response):
  return tokenizer.decode(generated_response[0])

In [13]:
model, tokenizer = model_formation("TheBloke/Mistral-7B-v0.1-AWQ")

def pipeline(model,tokenizer,prompt):
  tokens = tokenize(tokenizer=tokenizer, prompt=prompt)
  gen_out = generate_output(model, tokens)
  decode = decode_output(gen_out)
  return decode

config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

D:\Anaconda3\envs\TF2GPU\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\22830110\.cache\huggingface\hub\models--TheBloke--Mistral-7B-v0.1-AWQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

quant_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Fusing layers...: 100%|████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 61.41it/s]


In [14]:
pipeline(model=model,tokenizer=tokenizer,prompt='hi')

AssertionError: Torch not compiled with CUDA enabled

## Google Flan T5-XL

In [15]:
# pip install accelerate
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration


input_text = "translate English to German: How old are you?"

In [16]:
def model_and_tokenizer(model_path_or_name):
  tokenizer = T5Tokenizer.from_pretrained(model_path_or_name)
  model = T5ForConditionalGeneration.from_pretrained(model_path_or_name, device_map="auto", torch_dtype=torch.float16)
  return model, tokenizer

def make_inference(model,tokenizer,prompt):
  input_ids = tokenizer(prompt, return_tensors="pt",max_length=1000,truncation=True).input_ids.to("cuda")
  outputs = model.generate(input_ids)
  return tokenizer.decode(outputs[0],skip_special_tokens=True)

def pipeline_t5(prompt):
  model,tokenizer = model_and_tokenizer("google/flan-t5-xl")
  result = make_inference(model, tokenizer, prompt)
  return result

def t5_gradio_pipeline(message,history):
  model,tokenizer = model_and_tokenizer("google/flan-t5-xl")
  result = make_inference(model, tokenizer, message)
  return result

In [17]:
model, tokenizer = model_and_tokenizer("google/flan-t5-xl")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

D:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\22830110\.cache\huggingface\hub\models--google--flan-t5-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [18]:
make_inference(model,tokenizer,'translate English to German: How old are you?')

D:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Wie alt sind Sie?'

In [17]:
make_inference(model,tokenizer,'And what do you do for a living?')

"I'm a therapist."

In [19]:
import gradio as gr

In [ ]:
interface = gr.ChatInterface(fn=t5_gradio_pipeline)
interface.launch()

In [19]:
# Acquire the training data from Hugging Face
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,Seq2SeqTrainer
from datasets import load_dataset

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
DATA_NAME = "yahoo_answers_qa"
yahoo_answers_qa = load_dataset(DATA_NAME)
yahoo_answers_qa = yahoo_answers_qa["train"].train_test_split(test_size=0.3)

Generating train split:   0%|          | 0/87362 [00:00<?, ? examples/s]

In [20]:
# Check the length of the data and its structure
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 61153
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 26209
    })
})

In [21]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs


In [22]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)

Map:   0%|          | 0/61153 [00:00<?, ? examples/s]

Map:   0%|          | 0/26209 [00:00<?, ? examples/s]

D:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\transformers\models\t5\tokenization_t5.py:309: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [23]:
import nltk, evaluate
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [24]:
import numpy as np

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [25]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [26]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

D:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [28]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 21.00 GiB is allocated by PyTorch, and 2.01 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [21]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [27]:
torch. cuda. is_available()

True

In [25]:
import os 

In [30]:
os.getcwd()

'C:\\'

In [29]:
os. chdir('C:\\')

In [34]:
import shutil 
  
# Path 
path = "C:\\Users\\22830110"
  
# Get the disk usage statistics 
# about the given path 
stat = shutil.disk_usage(path) 
  
# Print disk usage statistics 
print("Disk usage statistics:") 
print(stat) 

Disk usage statistics:
usage(total=429496725504, used=339160104960, free=90336620544)


In [ ]:
!gi